# Part1

For many Trading Strategies, we need __historical Data__ (e.g. last 2 hours) to create Strategy Features. <br>

- Option 1: we could __wait__ until our __stream collects sufficient Data__ -> __not practicable__
- Option 2: we __pull historical data__ first and __merge__ with live data -> __best practice__

In [1]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
from datetime import datetime, timedelta

In [2]:
api_key = "TGV2OZgIKqiCEWvhVUAUDGBB99Eixo1z2ILlZ0h3lRdutfklTwRlqzPYpIazsYcn"
secret_key = "CrgLtKo6MRxdL6tJRNKokKPKOAWa0Rn59Q9KfX57yal0E1WTqz79jk2IT03g6VcZ"
#test api_key
test_api_Key = "ni1RGarrqKhXFMnmqZt2EEbEJkpGDXs4AxHml3r7aevzg7uqUR1rTFbNngWiTOln"
test_secret_Key = "HqFm7zQNQPOEJdYMgLho7engJbah6FAH98Eqf607h467CB9RM9e6dHDytnQbKGgk"

In [3]:
client = Client(api_key = test_api_Key, api_secret = test_secret_Key, tld = "com", testnet = True)

In [4]:
now = datetime.utcnow()
now

datetime.datetime(2024, 3, 26, 13, 47, 57, 534707)

In [5]:
past = now - timedelta(days = 2)
past

datetime.datetime(2024, 3, 24, 13, 47, 57, 534707)

In [6]:
str(past)

'2024-03-24 13:47:57.534707'

In [7]:
def get_most_recent(symbol, interval, days):
    
    now = datetime.utcnow()
    past = str(now - timedelta(days = days))
    
    bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                        start_str = past, end_str = None, limit = 1000)
    df = pd.DataFrame(bars)
    df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
    df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                  "Clos Time", "Quote Asset Volume", "Number of Trades",
                  "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    df.set_index("Date", inplace = True)
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors = "coerce")
    df["Complete"] = [True for row in range(len(df)-1)] + [False]
    
    return df

In [12]:
df = get_most_recent(symbol = "BTCUSDT", interval = "1m", days = 2)
df

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2024-03-24 15:33:00,65509.92,65509.94,65486.07,65486.61,0.13537,True
2024-03-24 15:34:00,65486.61,65487.44,65465.79,65487.44,0.18077,True
2024-03-24 15:35:00,65489.67,65542.70,65489.67,65542.70,0.27377,True
2024-03-24 15:36:00,65545.87,65559.49,65490.01,65523.79,0.16239,True
2024-03-24 15:37:00,65523.79,65560.00,65516.00,65534.38,0.25502,True
...,...,...,...,...,...,...
2024-03-26 15:28:00,70118.17,70119.40,70039.92,70118.87,0.16399,True
2024-03-26 15:29:00,70118.87,70119.72,69062.00,69721.31,0.19927,True
2024-03-26 15:30:00,69721.31,70096.00,69062.00,70087.53,0.38447,True


In [17]:
get_most_recent(symbol = "BTCUSDT", interval = "1m", days = 2)

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2024-03-24 15:35:00,65489.67,65542.70,65489.67,65542.70,0.27377,True
2024-03-24 15:36:00,65545.87,65559.49,65490.01,65523.79,0.16239,True
2024-03-24 15:37:00,65523.79,65560.00,65516.00,65534.38,0.25502,True
2024-03-24 15:38:00,65516.00,65582.00,65515.99,65572.00,0.36684,True
2024-03-24 15:39:00,65572.04,65609.28,65570.00,65609.28,0.27107,True
...,...,...,...,...,...,...
2024-03-26 15:30:00,69721.31,70096.00,69062.00,70087.53,0.38447,True
2024-03-26 15:31:00,70087.54,70096.00,69721.31,70007.70,0.26361,True
2024-03-26 15:32:00,70007.68,70052.00,70001.25,70001.25,0.20978,True


# Part2

In [18]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length):
        self.symbol = symbol
        self.bar_length = bar_length
        # self.data = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days) # NEW!
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days): # NEW!
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df # Create self.data
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print("Time: {} | Price: {}".format(event_time, close))
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]

In [19]:
trader = LongOnlyTrader(symbol = "BTCUSDT", bar_length = "1m")
trader

In [20]:
trader.start_trading(historical_days = 2)

Time: 2024-03-26 15:36:06.091000 | Price: 70179.25
Time: 2024-03-26 15:36:08.166000 | Price: 70181.09
Time: 2024-03-26 15:36:10.295000 | Price: 70189.89
Time: 2024-03-26 15:36:12.382000 | Price: 70184.01
Time: 2024-03-26 15:36:14.499000 | Price: 70184.0
Time: 2024-03-26 15:36:16.612000 | Price: 70167.99
Time: 2024-03-26 15:36:18.879000 | Price: 70146.91
Time: 2024-03-26 15:36:21.069000 | Price: 70146.9
Time: 2024-03-26 15:36:23.116000 | Price: 70141.1
Time: 2024-03-26 15:36:25.246000 | Price: 70167.5
Time: 2024-03-26 15:36:27.432000 | Price: 70184.0
Time: 2024-03-26 15:36:29.516000 | Price: 70176.32
Time: 2024-03-26 15:36:31.904000 | Price: 70178.01
Time: 2024-03-26 15:36:33.918000 | Price: 70194.01
Time: 2024-03-26 15:36:36.131000 | Price: 70200.0
Time: 2024-03-26 15:36:38.132000 | Price: 70200.0
Time: 2024-03-26 15:36:40.217000 | Price: 70200.0
Time: 2024-03-26 15:36:42.296000 | Price: 70200.0
Time: 2024-03-26 15:36:44.437000 | Price: 70210.77
Time: 2024-03-26 15:36:46.484000 | Price

In [21]:
trader.twm.stop()

Time: 2024-03-26 15:36:48.748000 | Price: 70218.01
Time: 2024-03-26 15:36:50.801000 | Price: 70212.01


In [22]:
trader.data

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2024-03-24 15:37:00,65523.79,65560.00,65516.00,65534.38,0.25502,True
2024-03-24 15:38:00,65516.00,65582.00,65515.99,65572.00,0.36684,True
2024-03-24 15:39:00,65572.04,65609.28,65570.00,65609.28,0.27107,True
2024-03-24 15:40:00,65604.01,65605.52,65593.71,65593.72,0.07345,True
2024-03-24 15:41:00,65593.72,65662.21,65593.72,65662.21,0.29758,True
...,...,...,...,...,...,...
2024-03-26 15:32:00,70007.68,70052.00,70001.25,70001.25,0.20978,True
2024-03-26 15:33:00,70008.00,70083.82,69791.10,70083.29,0.39553,True
2024-03-26 15:34:00,70083.82,70096.00,69062.00,70004.68,0.30464,True
